In [2]:
import shutil
import os

# Directory to delete
EXTRACT_DIR = 'test'

# Check if the directory exists and delete it
if os.path.isdir(EXTRACT_DIR):
    shutil.rmtree(EXTRACT_DIR)
    print(f"Directory '{EXTRACT_DIR}' has been deleted.")
else:
    print(f"Directory '{EXTRACT_DIR}' does not exist.")

Directory 'test' has been deleted.


In [3]:
import zipfile
import shutil
import os
import pandas as pd
print("CPU cores: ", os.cpu_count())

CPU cores:  144


In [4]:
def extract_zip(zip_path):

    if not zipfile.is_zipfile(zip_path):
        raise ValueError(f"The file at {zip_path} is not a valid ZIP archive.")
    
    # Determine the output directory name from the zip file name
    base_dir = os.path.dirname(zip_path)
    zip_filename = os.path.basename(zip_path)
    folder_name = os.path.splitext(zip_filename)[0]
    extract_to = os.path.join(base_dir, folder_name)

    # Create the directory if it doesn't exist
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    # Extract the ZIP file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(path=extract_to)

    return extract_to


if __name__ == "__main__":
    zip_file_path = '0.zip' 
    extracted_dir = extract_zip(zip_file_path)
    print(f"Extracted to: {extracted_dir}")

Extracted to: 0


In [5]:
leak_directory = "0"

# Path to the __MACOSX folder
macosx_folder = os.path.join(leak_directory, "__MACOSX")

# Check if __MACOSX exists and remove it - creates issues when analyzing the data, and its not needed, made automatically by MacOS
if os.path.exists(macosx_folder) and os.path.isdir(macosx_folder):
    shutil.rmtree(macosx_folder)
    print(f"Deleted - {macosx_folder}")
else:
    print(f"Folder not found - {macosx_folder}")

# Organize files by extension into subfolders
for root, dirs, files in os.walk(leak_directory):
    for file in files:
        # Skip hidden files and __MACOSX if any reappear
        if file.startswith('.') or '__MACOSX' in root:
            continue

        # Get the file extension (in lowercase, without the dot)
        file_extension = os.path.splitext(file)[1].lower().lstrip('.')
        if not file_extension:
            file_extension = "no_extension"

        # Define the new subfolder path
        subfolder_path = os.path.join(leak_directory, file_extension)

        # Create the subfolder if it doesn't exist
        os.makedirs(subfolder_path, exist_ok=True)

        # Define source and destination paths
        source_path = os.path.join(root, file)
        destination_path = os.path.join(subfolder_path, file)

        # Move the file if source and destination are not the same
        if os.path.abspath(source_path) != os.path.abspath(destination_path):
            shutil.move(source_path, destination_path)

# Remove any empty folders within the parent directory
for dirpath, dirnames, filenames in os.walk(leak_directory, topdown=False):
    if not dirnames and not filenames:
        try:
            os.rmdir(dirpath)
            print(f"Removed empty folder: {dirpath}")
        except OSError:
            pass  

Deleted - 0/__MACOSX


In [6]:
# Delete Duplicate of 0 which is empty

# Directory to delete
EXTRACT_DIR = '0/0'

# Check if the directory exists and delete it
if os.path.isdir(EXTRACT_DIR):
    shutil.rmtree(EXTRACT_DIR)
    print(f"Directory '{EXTRACT_DIR}'deleted")
else:
    print(f"Directory '{EXTRACT_DIR}' does not exist")

Directory '0/0'deleted


# **DATA PARSING**

# Dataframe with all file location

In [7]:
def get_folder_file_dataframe(root_dir):
    folders = []
    for file_name in os.listdir(root_dir):
        full_path = os.path.join(root_dir, file_name)
        if os.path.isdir(full_path):
            folders.append(file_name)

    series_list = []
    for folder in folders:
        folder_path = os.path.join(root_dir, folder)
        files = [
            os.path.join(folder_path, file)
            for file in os.listdir(folder_path)
            if os.path.isfile(os.path.join(folder_path, file)) and not file.startswith('.')
        ]
        s = pd.Series(files, name=folder)
        series_list.append(s)

    df = pd.concat(series_list, axis=1)
    return df

In [8]:
df = get_folder_file_dataframe(leak_directory)
df.head(30)       

,md,png,log,txt
0,0/md/dbc9c90e-a3e6-4d71-bb93-5fb8394095ac.md,0/png/64bba692-d430-440c-9f1e-2575f45770af_6.png,0/log/77010155050.log,0/txt/IDNET.txt
1,0/md/28.md,0/png/12756724-394c-4576-b373-7c53f1abbd94_0.png,0/log/77753527617.log,0/txt/IDTV.txt
2,0/md/5.md,0/png/f179eb06-0c53-44df-a13f-570be23355bb_1.png,0/log/tele2-lbs.log,0/txt/beeline-77774042222.txt
3,0/md/38.md,0/png/5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_24.png,0/log/tele2-cdr.log,0/txt/beeline-77051056626.txt
4,0/md/9d7bc879-3250-4013-ac04-5ff9bd6dff40.md,0/png/5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_30.png,0/log/tele2-crm.log,0/txt/beeline-crm.txt
5,0/md/18.md,0/png/5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_18.png,0/log/77783030133.log,0/txt/UBSCRIBER.txt
6,0/md/9fd06037-11f1-4ad5-9a7d-cbfb3fa4193b.md,0/png/0-adaf869e-920a-4a17-91bd-e2ef3125c10e.png,NaN,0/txt/beeline-cdr.txt
7,0/md/3348953d-66e9-4cac-8675-65bb5f2ef929.md,0/png/5387a301-0af8-4e24-a197-20189f87b9ef_8.png,NaN,0/txt/CRM.txt
8,0/md/1.md,0/png/0-32eb7662-f212-4811-a7c1-1cfeb121cd99.png,NaN,0/txt/LAC.txt
9,0/md/19.md,0/png/912204cb-8ab7-48b8-9abf-d803f3804d08_11.png,NaN,0/txt/beeline-lbs.txt


# Use LLM to classify the source files

In [9]:
import os
import concurrent.futures
import pandas as pd
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from tqdm import tqdm

In [10]:
llm = Ollama(model="llama3.1:8b")

prompt_template = PromptTemplate(
    input_variables=["content"],
    template="""
You are analyzing the content of a file.

File content:
\"\"\"
{content}
\"\"\"

1. Classify the content into one of the following categories ONLY: chats, images, other.
2. State your confidence in the classification as one of: high, medium, or low.
3. I will have to make a csv, please give me a list of headers based on the content. E.g. "[<header_name>, <heaer_name2>, etc.]
Respond in the following format:
Category: <chats|images|other>
Confidence: <high|medium|low>
"""
)

chain = LLMChain(llm=llm, prompt=prompt_template)

# Preprocess the files to the first 20 lines to shorten analysis times
def preprocess_first_20_lines(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            lines = [line.strip() for _, line in zip(range(20), f)]
            content = " ".join(lines)
        return file_path, content
    except Exception:
        return file_path, ""

# Use only the first column of the DataFrame
first_column = df.columns[0]
file_paths = df[first_column].dropna().unique().tolist()

# Preprocess the file for faster classification
with concurrent.futures.ThreadPoolExecutor() as executor:
    file_data = list(executor.map(preprocess_first_20_lines, file_paths))

# Classify and collect by category
valid_categories = {"chats", "images", "other"}
categorized_files = {cat: [] for cat in valid_categories}

for file_path, content in tqdm(file_data, desc="Classifying files"):
    if not content:
        continue

    try:
        response = chain.run(content=content).strip().lower()
        lines = response.splitlines()

        category = ""
        for line in lines:
            if line.startswith("category:"):
                category = line.replace("category:", "").strip()
                break
        if category not in valid_categories:
            category = "other"

        categorized_files[category].append(file_path)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Convert to DataFrame (columns = categories)
# Determine the maximum list length among all categories
max_len = 0
for file_list in categorized_files.values():
    if len(file_list) > max_len:
        max_len = len(file_list)

# Pad each list with None to match max length
padded = {}
for category, file_list in categorized_files.items():
    padding_needed = max_len - len(file_list)
    padded[category] = file_list + [None] * padding_needed
result_df = pd.DataFrame(padded)
result_df.to_csv("classified_by_category.csv", index=False)
result_df.head()

/tmp/ipykernel_313/23640893.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
Classifying files:   0%|          | 0/70 [00:00<?, ?it/s]/tmp/ipykernel_313/23640893.py:51: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = chain.run(content=content).strip().lower()
Classifying files: 100%|██████████| 70/70 [03:58<00:00,  3.40s/it] 


,other,images,chats
0,0/md/9d7bc879-3250-4013-ac04-5ff9bd6dff40.md,0/md/dbc9c90e-a3e6-4d71-bb93-5fb8394095ac.md,0/md/28.md
1,None,0/md/9fd06037-11f1-4ad5-9a7d-cbfb3fa4193b.md,0/md/5.md
2,None,0/md/3348953d-66e9-4cac-8675-65bb5f2ef929.md,0/md/38.md
3,None,0/md/07f179c5-5705-4dbd-94a7-66eed1e066b0.md,0/md/18.md
4,None,0/md/01cdc26f-e773-4ad7-8808-d04abf16aae7.md,0/md/1.md


In [11]:
import os
import pandas as pd
import re
from collections import defaultdict

# Script to check for cross refernced files

In [12]:
import pandas as pd
import os
import re
from collections import defaultdict

In [13]:
# Extract all unique file paths from the DataFrame 
file_paths = df.stack().dropna().unique().tolist()

# Read file content
def read_file(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return f.read()
    except:
        return "Failed to open file"

# Read file contents into a dictionary 
file_contents = {path: read_file(path) for path in file_paths}

# Build a reference map (which files mention which) 
reference_map = defaultdict(list)

for source_path, content in file_contents.items():
    if content is None:
        continue  # skip if content is None

    for target_path in file_paths:
        if target_path == source_path:
            continue  # skip comparing file to itself

        target_filename = os.path.basename(target_path)
        pattern = re.escape(target_filename)

        if re.search(rf'\b{pattern}\b', content):
            reference_map[source_path].append(target_path)

# Convert reference map to a DataFrame
ref_df = pd.DataFrame([
    {"source_file": src, "mentions": tgt}
    for src, tgts in reference_map.items()
    for tgt in tgts
])

# Group mentions into lists per source_file
ref_summary = ref_df.groupby("source_file")["mentions"].apply(list).reset_index()

# Some source_file names are also in mentions, therefore they aren't a source file anymore
source_files = set(ref_summary["source_file"])
mentioned_files = set(file for mention_list in ref_summary["mentions"] for file in mention_list)
common_files = source_files.intersection(mentioned_files)

# Filter out common files from the DataFrame
filtered_ref_summary = ref_summary[~ref_summary["source_file"].isin(common_files)].reset_index(drop=True)

filtered_ref_summary.to_csv("file_reference_map.csv", index=False)
print(filtered_ref_summary.head(10))
print("Unique file paths in original DataFrame:", df.stack().nunique())

                                    source_file  \
0  0/md/01cdc26f-e773-4ad7-8808-d04abf16aae7.md   
1                                    0/md/10.md   
2                                    0/md/13.md   
3                                    0/md/15.md   
4  0/md/178e3898-903d-47cf-bfbe-061e7dc18895.md   
5                                    0/md/19.md   
6                                     0/md/2.md   
7                                    0/md/20.md   
8                                    0/md/21.md   
9                                    0/md/22.md   

                                            mentions  
0  [0/png/01cdc26f-e773-4ad7-8808-d04abf16aae7_1_...  
1  [0/png/0-6bcc0131-e4ad-421e-bb1f-d8ebe5eeec7b....  
2  [0/png/0-adaf869e-920a-4a17-91bd-e2ef3125c10e....  
3  [0/png/0-b8cea3b1-4dde-4438-9b1a-6faf690bbad0....  
4  [0/png/178e3898-903d-47cf-bfbe-061e7dc18895_8....  
5  [0/png/6848748d-2881-4c26-b153-fcd5373d2f1c.pn...  
6  [0/md/07f179c5-5705-4dbd-94a7-66eed1e066b0.md,... 

# Check for more linkages in the files + add the missing source files from the chats

In [14]:
import pandas as pd
import os
import re
from ast import literal_eval
import ast

In [15]:
def is_readable_text_file(path):
    return any(path.endswith(ext) for ext in ['txt', 'md', 'rst', 'tex', 'nfo', 'readme', 'rtf', 'doc', 'docx',
    'cfg', 'conf', 'config', 'ini', 'json', 'yaml', 'yml', 'toml',
    'log', 'lst', 'cnf', 'properties', 'prefs',
    'csv', 'tsv', 'dat', 'db', 'dbf', 'sql', 'xml',
    'ssv', 'psv', 'jsonl', 'parquet', 'orc',
    'html', 'htm', 'xhtml', 'xht', 'css', 'js',
    'jsx', 'ts', 'tsx', 'vue', 'erb', 'ejs', 'jsp',
    'liquid', 'handlebars', 'hbs', 'mustache',
    'py', 'pyw', 'ipynb', 'java', 'c', 'cpp', 'h', 'hpp', 'cs',
    'sh', 'bash', 'zsh', 'ksh', 'bat', 'cmd', 'ps1',
    'r', 'jl', 'pl', 'pm', 'rb', 'go', 'lua', 'php',
    'swift', 'scala', 'dart', 'asm', 'groovy',
    'rmd', 'sage', 'nb',
    'env', 'gradle', 'makefile', 'mak', 'mk',
    'dockerfile', 'gitignore', 'gitattributes', 'gitmodules',
    'cmake', 'make', 'ninja', 'build',
    'manifest', 'manifest.json', 'vtt', 'srt', 'resx', 'strings',
    'lang', 'po', 'mo', 'pot', 'msg', 'textbundle',
    'rego', 'tf', 'tfvars', 'cue', 'bzl', 'bazel', 'nix', 'dhall',
    'adoc', 'asciidoc', 'creole', 'mediawiki', 'wiki', 'org',
    'eml', 'msg', 'mbox', 'mail', 'ics', 'vcf'])

mentions2_list = []

# Iterate over each row of the summary
for _, row in filtered_ref_summary.iterrows():
    mention_dict = {}

    # For each mentioned file read file contents and create a dictionary key: location - values: file contents
    for mentioned_file in row["mentions"]:
        if not is_readable_text_file(mentioned_file):
            continue  # Skip unreadable formats

        # Read file content
        content = read_file(mentioned_file)
        if content in [None, "Failed to open file"]:
            continue

        # Search for any file-type references using regex
        # Match anything with a name and known file extension
        found_files = re.findall(r'[\w\-/]*\.\w+', content)

        # Filter duplicates and add to the mention_dict
        if found_files:
            mention_dict[mentioned_file] = list(set(found_files))

    mentions2_list.append(mention_dict)

# Add this as a new column
filtered_ref_summary["mentions2"] = mentions2_list

# Get the list of chat files from result_df - LLM clasification
chat_files = result_df['chats'].dropna().unique()

# Get the list of source files already in filtered_ref_summary
source_files = filtered_ref_summary['source_file'].dropna().unique()

# Find missing chat files and collect chat files that are not found in source files
existing_source_files = set(source_files)
missing_chat_files = []
for chat_file in chat_files:
    if chat_file not in existing_source_files:
        missing_chat_files.append(chat_file)

# Create a new DataFrame with empty mentions and linkage columns
new_rows = pd.DataFrame({
    'source_file': missing_chat_files,
})

# Append the new rows to ref_summary
filtered_ref_summary = pd.concat([filtered_ref_summary, new_rows], ignore_index=True)
        
# Save the updated DataFrame to CSV
filtered_ref_summary.to_csv("file_reference_map_extended.csv", index=False)
print(filtered_ref_summary.columns)

Index(['source_file', 'mentions', 'mentions2'], dtype='object')


# Structuring the Data into CSV - Step 1 all MD files

In [16]:
import os
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup

In [17]:
# Create a folder and dumb all csvs
output_dir = "csvs"
os.makedirs(output_dir, exist_ok=True)

# Get all MD files from the DataFrame
all_file_paths = pd.unique(result_df.values.ravel('K'))
md_files = []
for file_path in all_file_paths:
    if isinstance(file_path, str) and file_path.endswith(".md") and os.path.exists(file_path):
        md_files.append(file_path)
        
# Use the LLM's clasification of MD files
chat_files = set(result_df['chats'].dropna().astype(str))

# Extract visible text + ANY file reference 
def extract_text_and_files(td):
    text = td.get_text(strip=True)
    links = []
    for tag in td.find_all(['a', 'img']):
        link = tag.get('href') or tag.get('src')
        if link:
            fname = os.path.basename(link.strip("'\""))
            links.append(fname)
    return text + (" " + " ".join(links) if links else "")

# This method extracts chat table from HTML
def process_chat_md(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
        soup = BeautifulSoup(content, "lxml")
        rows = soup.find_all("tr")[1:]
        data = []
        for row in rows:
            cols = row.find_all("td")
            if len(cols) == 4:
                data.append({
                    "Time": cols[0].get_text(strip=True),
                    "From": cols[1].get_text(strip=True),
                    "To": cols[2].get_text(strip=True),
                    "Message": extract_text_and_files(cols[3])
                })
        return pd.DataFrame(data) if data else None
    except Exception as e:
        print(f"Error processing chat file {file_path}: {e}")
        return None

# Extract the contents of the other file types that the LLM classified
def process_reference_md(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
        soup = BeautifulSoup(content, "lxml")
        matches = []
        for tag in soup.find_all(['a', 'img']):
            link = tag.get('href') or tag.get('src')
            if link:
                fname = os.path.basename(link.strip("'\""))
                if not fname.startswith("0-"):
                    matches.append(fname)
        matches = list(set(matches))
        return pd.DataFrame([{"File_Name": f} for f in matches]) if matches else None
    except Exception as e:
        print(f"Error processing reference file {file_path}: {e}")
        return None

# Loop through all MD filesand call the above methods for the afferent file type
for file_path in md_files:
    csv_name = os.path.basename(file_path).replace(".md", ".csv")
    csv_path = os.path.join(output_dir, csv_name)

    if not os.path.exists(csv_path):
        if file_path in chat_files:
            df = process_chat_md(file_path)
        else:
            df = process_reference_md(file_path)

        if df is not None:
            df.to_csv(csv_path, index=False)
        else:
            print(f"Couldn't save to CSV: {file_path}")

In [18]:
import os
import ast
from pathlib import Path

In [19]:
csv_dir = "csvs"

# Replace MD with CSV if the corresponding file exists
def replace_md_with_csv(item):
    if isinstance(item, str) and item.endswith(".md"):
        base_name = Path(item).stem  # .stem extracts the filename with no extension
        # CSV search
        csv_path = os.path.join(csv_dir, f"{base_name}.csv")
        if os.path.exists(csv_path):
            return csv_path
        else:
            print(f"CSV {item} not found")
    return item

# Recursively handle strings, lists, and dictionaries
def process_column_cell(cell):
    if isinstance(cell, str):
        return replace_md_with_csv(cell)
    
    elif isinstance(cell, list):
        new_list = []
        for item in cell:
            new_item = process_column_cell(item)
            new_list.append(new_item)
        return new_list

    elif isinstance(cell, dict):
        new_dict = {}
        for key, value in cell.items():
            new_key = replace_md_with_csv(key)
            new_value = process_column_cell(value)
            new_dict[new_key] = new_value
        return new_dict

    else:
        return cell
    
for col in filtered_ref_summary.columns:
    filtered_ref_summary[col] = filtered_ref_summary[col].apply(process_column_cell)

filtered_ref_summary.to_csv("filtered_ref_summary_csv_replaced.csv", index=False)
filtered_ref_summary.head()        

,source_file,mentions,mentions2
0,csvs/01cdc26f-e773-4ad7-8808-d04abf16aae7.csv,[0/png/01cdc26f-e773-4ad7-8808-d04abf16aae7_1_...,{}
1,csvs/10.csv,[0/png/0-6bcc0131-e4ad-421e-bb1f-d8ebe5eeec7b....,{'csvs/12756724-394c-4576-b373-7c53f1abbd94.cs...
2,csvs/13.csv,[0/png/0-adaf869e-920a-4a17-91bd-e2ef3125c10e....,{'csvs/585875ff-f8c5-4a02-acd7-fef37dc9ff11.cs...
3,csvs/15.csv,[0/png/0-b8cea3b1-4dde-4438-9b1a-6faf690bbad0....,{}
4,csvs/178e3898-903d-47cf-bfbe-061e7dc18895.csv,[0/png/178e3898-903d-47cf-bfbe-061e7dc18895_8....,{}


# Structuring the Data into CSV - Step 2 OCR on image files

LIST OF FILES FOR TEST: 5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_0.png, 08a6bcd3-6477-4252-8f35-4f8f80d114f9.png, 1afcf93d-50f1-4f1e-896d-87b0da7519f7.png, 32eb7662-f212-4811-a7c1-1cfeb121cd99.png, 6bcc0131-e4ad-421e-bb1f-d8ebe5eeec7b.png, 79d9b7f2-cfe4-4615-9b75-8fea33fc0c9d.png, 4ae9bf34-c16c-4684-aa92-fec65a151275.png, 5a84cde3-7175-4044-8c88-d4c883a8fd38.png, e705d192-90ee-4fd1-9dcd-061958d1817f.png, 912204cb-8ab7-48b8-9abf-d803f3804d08_4.png

# **TEST-1: Extraction with LLAVA**

In [20]:
import base64
import requests
import os

# List of image file paths
image_paths = [
    "0/png/5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_0.png", 
    "0/png/08a6bcd3-6477-4252-8f35-4f8f80d114f9.png", 
    "0/png/1afcf93d-50f1-4f1e-896d-87b0da7519f7.png", 
    "0/png/32eb7662-f212-4811-a7c1-1cfeb121cd99.png", 
    "0/png/6bcc0131-e4ad-421e-bb1f-d8ebe5eeec7b.png", 
    "0/png/79d9b7f2-cfe4-4615-9b75-8fea33fc0c9d.png", 
    "0/png/4ae9bf34-c16c-4684-aa92-fec65a151275.png", 
    "0/png/5a84cde3-7175-4044-8c88-d4c883a8fd38.png", 
    "0/png/e705d192-90ee-4fd1-9dcd-061958d1817f.png", 
    "0/png/912204cb-8ab7-48b8-9abf-d803f3804d08_4.png"
]

# Define the prompt
prompt = "EXTRACT AND ONLY GIVE ME THE Chinese text in the given image. NO TRANSLATION, ONLY extract the TEXT IN CHINESE MANDARIN FROM THE PNG FILE"

# Process each image
for image_path in image_paths:
    with open(image_path, "rb") as f:
        image_b64 = base64.b64encode(f.read()).decode("utf-8")

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "llava:13b",
            "prompt": prompt,
            "images": [image_b64],
            "stream": False
        },
    )

    filename = os.path.basename(image_path)
    if response.ok:
        output_text = response.json()["response"]
    else:
        output_text = f"Error: {response.text}"

    print(f"{filename}:\n{output_text.strip()}\n")

5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_0.png:
The Chinese text on the image is: "未来医疗技术与科学研究开展方向".

08a6bcd3-6477-4252-8f35-4f8f80d114f9.png:
I'm unable to process images or provide translations directly from this interface. If you need assistance with the Chinese text in the image, please type out the text in a separate message, and I will be happy to help you with its translation or any other question you may have!

1afcf93d-50f1-4f1e-896d-87b0da7519f7.png:
你好！我有一个问题想要请教。我现在正在做一个简单的项目，需要购买一些电子设备。如果可以的话，能否向您请教一下价格和采购建议？谢谢！

32eb7662-f212-4811-a7c1-1cfeb121cd99.png:
The image you've provided shows a screenshot of a computer interface with several lines of Chinese text. Here is the extracted Chinese text from the visible parts of the screen:

```
发送人：陈某某
发送时间：2014年6月8日 下午9点37分
标题：关于一些信息的提供
内容：

这是一个测试文本。请问我应该在哪里写？
```

Please note that the text may be incomplete as it only covers part of what is visible in the image.

6bcc0131-e4ad-421e-bb1f-d8ebe5eeec7b.png:
抱歉，由于此图像的质量不够高，我无法识别并提取中文。请提供

# **Data has been added from MacOS OCR**

# *Changing the files in the chats with the extracted output*

In [ ]:
# Go through the csv file and for each file.md or image.png replace with the OCR output from the folder

# **DATA TRANSLATION Verification - 3 LLMs**

In [21]:
import os
import pandas as pd

In [22]:
# Use the csvs folder and the LLM classification of MD files to create a new df that stores the csvs into categories.

csv_folder = "csvs"

csv_mapping = {}
# Loop through each file in the CSV folder
for filename in os.listdir(csv_folder):
    # Check for csv
    if filename.endswith('.csv'):
        # Replace the ".csv" extension with ".md" to create the key
        md_filename = filename.replace('.csv', '.md')
        
        # Set the value to the full path to the CSV file 
        csv_full_path = os.path.join(csv_folder, filename)
        
        # Add the key-value pair to the mapping
        csv_mapping[md_filename] = csv_full_path

# Step 2: Define function to replace .md path with .csv path
def replace_md_with_csv(val):
    if isinstance(val, str):
        base = os.path.basename(val)
        return csv_mapping.get(base, val)
    return val

df_csvs_llm = result_df.applymap(replace_md_with_csv)
df_csvs_llm.to_csv("updated_with_csv_paths.csv", index=False)
print(df_csvs_llm.head())

                                           other  \
0  csvs/9d7bc879-3250-4013-ac04-5ff9bd6dff40.csv   
1                                           None   
2                                           None   
3                                           None   
4                                           None   

                                          images        chats  
0  csvs/dbc9c90e-a3e6-4d71-bb93-5fb8394095ac.csv  csvs/28.csv  
1  csvs/9fd06037-11f1-4ad5-9a7d-cbfb3fa4193b.csv   csvs/5.csv  
2  csvs/3348953d-66e9-4cac-8675-65bb5f2ef929.csv  csvs/38.csv  
3  csvs/07f179c5-5705-4dbd-94a7-66eed1e066b0.csv  csvs/18.csv  
4  csvs/01cdc26f-e773-4ad7-8808-d04abf16aae7.csv   csvs/1.csv  


# **TEST-1: gemma3:27b**

In [9]:
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

csv_path = 'csvs/10.csv'
output_path = '3llms_10.csv'  # Output in project root directory
df = pd.read_csv(csv_path)

def get_context(df, index, window=5):
    start = max(0, index - window)
    end = min(len(df), index + window + 1)
    return "\n".join(df['Message'].iloc[start:end].dropna())

def query_ollama(prompt, model="gemma3:27b"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload, timeout=90)
        response.raise_for_status()
        return response.json()["response"].strip()
    except Exception as e:
        print(f"Ollama error: {e}")
        return "[Translation Error]"

def translate_row(index):
    target = df.at[index, 'Message']
    if pd.isna(target):
        return index, ""
    context = get_context(df, index)
    prompt = (
        "You are translating Chinese messages to English. Below is a series of related messages. "
        "Use the full context to understand the meaning, but only translate the specific message provided.\n\n"
        f"Context:\n{context}\n\n"
        f"Message to translate:\n{target}\n\n"
        "ONLY RETURN the English translation of the message, ANYTHING ELSE IS FORBIDDEN! "
        "If you encounter a filename or file reference, preserve it exactly as it appears, DON'T add anything!"
    )
    translation = query_ollama(prompt)
    return index, translation

translations = [""] * len(df)

futures = []
with ThreadPoolExecutor(max_workers=8) as executor:
    for index in df.index:
        futures.append(executor.submit(translate_row, index))

    for future in tqdm(as_completed(futures), total=len(futures), desc="Translating with Ollama"):
        idx, result = future.result()
        translations[idx] = result

translated_df = pd.DataFrame({'gemma3:27b': translations})
translated_df.to_csv(output_path, index=False)

Translating with Ollama: 100%|██████████| 149/149 [07:36<00:00,  3.06s/it]


# **TEST-2: 7shi/llama-translate:8b-q4_K_M**

In [10]:
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

source_csv_path = 'csvs/10.csv'
existing_translation_path = '3llms_10.csv'
new_model_name = '7shi/llama-translate:8b-q4_K_M'

# Load original messages and previously translated file
source_df = pd.read_csv(source_csv_path)
translated_df = pd.read_csv(existing_translation_path)

def get_context(df, index, window=5):
    start = max(0, index - window)
    end = min(len(df), index + window + 1)
    return "\n".join(df['Message'].iloc[start:end].dropna())

def query_ollama(prompt, model):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload, timeout=90)
        response.raise_for_status()
        return response.json()["response"].strip()
    except Exception as e:
        print(f"Ollama error: {e}")
        return "[Translation Error]"

def translate_row(index):
    target = source_df.at[index, 'Message']
    if pd.isna(target):
        return index, ""
    context = get_context(source_df, index)
    prompt = (
        "Transalte Mandarin to English. Below is a series of related messages. "
        f"Context:\n{context}\n\n"
        f"Message to translate:\n{target}\n\n"
    )
    translation = query_ollama(prompt, model=new_model_name)
    return index, translation

# Prepare for parallel processing
translations = [""] * len(source_df)
futures = []
with ThreadPoolExecutor(max_workers=8) as executor:
    for index in source_df.index:
        futures.append(executor.submit(translate_row, index))

    for future in tqdm(as_completed(futures), total=len(futures), desc=f"Translating with {new_model_name}"):
        idx, result = future.result()
        translations[idx] = result

translated_df[new_model_name] = translations
translated_df.to_csv(existing_translation_path, index=False)

Translating with 7shi/llama-translate:8b-q4_K_M: 100%|██████████| 149/149 [08:52<00:00,  3.57s/it]


# **TEST-3: yi:34b**

In [11]:
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# File paths
source_csv_path = 'csvs/10.csv'
existing_translation_path = '3llms_10.csv'
new_model_name = 'yi:34b'

# Load original source messages and existing translations
source_df = pd.read_csv(source_csv_path)
translated_df = pd.read_csv(existing_translation_path)

# Context window function
def get_context(df, index, window=5):
    start = max(0, index - window)
    end = min(len(df), index + window + 1)
    return "\n".join(df['Message'].iloc[start:end].dropna())

def query_ollama(prompt, model):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload, timeout=90)
        response.raise_for_status()
        return response.json()["response"].strip()
    except Exception as e:
        print(f"Ollama error: {e}")
        return "[Translation Error]"

# Translate a single row
def translate_row(index):
    target = source_df.at[index, 'Message']
    if pd.isna(target):
        return index, ""
    context = get_context(source_df, index)
    prompt = (
        "You are translating Chinese messages to English. Below is a series of related messages. "
        "Use the full context to understand the meaning, but only translate the specific message provided.\n\n"
        f"Context:\n{context}\n\n"
        f"Message to translate:\n{target}\n\n"
        "ONLY RETURN the English translation of the message, ANYTHING ELSE IS FORBIDDEN! "
        "If you encounter a filename or file reference, preserve it exactly as it appears, DON'T add anything!"
    )
    translation = query_ollama(prompt, model=new_model_name)
    return index, translation

# Run translations in parallel
translations = [""] * len(source_df)
futures = []
with ThreadPoolExecutor(max_workers=8) as executor:
    for index in source_df.index:
        futures.append(executor.submit(translate_row, index))

    for future in tqdm(as_completed(futures), total=len(futures), desc=f"Translating with {new_model_name}"):
        idx, result = future.result()
        translations[idx] = result

translated_df[new_model_name] = translations
translated_df.to_csv(existing_translation_path, index=False)

Translating with yi:34b: 100%|██████████| 149/149 [07:20<00:00,  2.96s/it]


# **Levenshtein Distance Between the 3 Translations**

In [14]:
import pandas as pd
import Levenshtein
from itertools import combinations

csv_path = '3llms_10.csv'
df = pd.read_csv(csv_path)

# Identify all translation columns 
translation_columns = [col for col in df.columns if not col.startswith('Levenshtein')]

# Compute Levenshtein distance for all possible combinations
for col1, col2 in combinations(translation_columns, 2):
    new_col_name = f"Levenshtein ({col1} vs {col2})"
    df[new_col_name] = [
        Levenshtein.distance(str(a), str(b)) if pd.notna(a) and pd.notna(b) else None
        for a, b in zip(df[col1], df[col2])
    ]

# Save updated DataFrame back to the same CSV
df.to_csv(csv_path, index=False)

# **CHATS TRANSLATION - GEMMA3:27B**

In [23]:
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Chats - CSV loop
for csv_path in df_csvs_llm['chats']:
    print(f"Processing: {csv_path}")
    
    # Load the individual CSV file
    df = pd.read_csv(csv_path)
    df['message_translation'] = ""

    # Function to get context around the target message
    def get_context(df, index, window=5):
        start = max(0, index - window)
        end = min(len(df), index + window + 1)
        return "\n".join(df['Message'].iloc[start:end].dropna())

    def query_ollama(prompt, model="gemma3:27b"):
        url = "http://localhost:11434/api/generate"
        payload = {
            "model": model,
            "prompt": prompt,
            "stream": False
        }
        try:
            response = requests.post(url, json=payload, timeout=90)
            response.raise_for_status()
            return response.json()["response"].strip()
        except Exception as e:
            print(f"Ollama error: {e}")
            return "[Translation Error]"

    # Function to process one row
    def translate_row(index):
        target = df.at[index, 'Message']
        if pd.isna(target):
            return index, ""
        context = get_context(df, index)
        prompt = (
            "You are translating Chinese messages to English. Below is a series of related messages. "
            "Use the full context to understand the meaning, but only translate the specific message provided.\n\n"
            f"Context:\n{context}\n\n"
            f"Message to translate:\n{target}\n\n"
            "ONLY RETURN the English translation of the message, ANYTHING ELSE IS FORBIDDEN! "
            "If you encounter a filename or file reference, preserve it exactly as it appears, DON'T add anything!"
        )
        translation = query_ollama(prompt)
        return index, translation

    # Run translations in parallel
    futures = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        for index in df.index:
            futures.append(executor.submit(translate_row, index))

        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Translating: {csv_path}"):
            idx, result = future.result()
            df.at[idx, 'message_translation'] = result

    # Save updated DataFrame back to the original file
    df.to_csv(csv_path, index=False)
    print(f"Finished and saved: {csv_path}\n")

Processing: csvs/28.csv


Translating: csvs/28.csv: 100%|██████████| 46/46 [02:55<00:00,  3.81s/it]


Finished and saved: csvs/28.csv

Processing: csvs/5.csv


Translating: csvs/5.csv: 100%|██████████| 17/17 [00:24<00:00,  1.46s/it]


Finished and saved: csvs/5.csv

Processing: csvs/38.csv


...

Finished and saved: csvs/7.csv

Processing: csvs/6.csv


Translating: csvs/6.csv: 100%|██████████| 136/136 [03:08<00:00,  1.38s/it]


Finished and saved: csvs/6.csv

Processing: csvs/2.csv


Translating: csvs/2.csv: 100%|██████████| 500/500 [16:03<00:00,  1.93s/it]

Finished and saved: csvs/2.csv



In [5]:
import pandas as pd

# Load the original CSV
df = pd.read_csv("csvs/10.csv")

# Print all column names to verify exact labels
print("CSV Columns:")
print(df.columns.tolist())

# Drop the specified columns (only if they exist)
columns_to_drop = ["message_translation", "financial_detection"]
df_cleaned = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Overwrite the original file
df_cleaned.to_csv("csvs/10.csv", index=False)  # Overwriting the same file

CSV Columns:
['Time', 'From', 'To', 'Message']


# **USER ANALYSIS**

In [4]:
import pandas as pd
import os
from collections import Counter

In [8]:
# Load the CSV file with paths
df_csvs = pd.read_csv("updated_with_csv_paths.csv")

def count_messages_per_user_from_csv(csv_path):
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f"Error reading {csv_path}: {e}")
        return Counter()

    # Normalize column names
    df.columns = [col.lower() for col in df.columns]

    # Identify the 'from' column
    from_columns = [col for col in df.columns if col in ['from', 'sender', 'user']]
    if not from_columns:
        return Counter()

    from_col = from_columns[0]

    # Drop missing senders and count messages
    user_counts = df[from_col].dropna().astype(str).str.strip()
    return Counter(user_counts[user_counts != ''])

# Initialize a counter
all_message_counts = Counter()

# Loop through each CSV path in the 'chats' column
for csv_path in df_csvs['chats'].dropna().unique():
    if isinstance(csv_path, str) and csv_path.endswith('.csv') and os.path.exists(csv_path):
        counts = count_messages_per_user_from_csv(csv_path)
        all_message_counts.update(counts)

# Convert to sorted list and display
sorted_user_message_counts = sorted(all_message_counts.items(), key=lambda x: x[1], reverse=True)

print("Messages sent per user across all chat logs:")
for user, count in sorted_user_message_counts:
    print(f"- {user}: {count}")

Messages sent per user across all chat logs:
- lengmo: 4981
- Shutd0wn: 3675
- wxid_5390224027312: 1409
- wxid_7p054rmzkhqf21: 894
- wxid_wh6x59w70y3r22: 620
- gzp1991101: 603
- wxid_zb45i0rc71yk21: 590
- wxid_70w3p1jin84k22: 523
- wxid_hlmnhsq64tt722: 483
- adpw90: 235
- qq78263462: 225
- wxid_12n748um1thl21: 192
- wxid_mgh25nentc4u22: 190
- just910420: 154
- wxid_c9yv0nsla3yn22: 116
- nullroot: 103
- wxid_icges6alg8cl21: 91
- wei592628: 87
- wxid_soekgggwnfgm21: 79
- wxid_nv9bv435fz3722: 77
- wxid_xusilpfkh31g21: 61
- ken73224: 58
- hack05112: 45
- tianyi-0608: 45
- wxid_kbys0kvzj4ta12: 38
- SWEET5683yao: 35
- wxid_zbytkn4qjl3r22: 29
- wxid_blw54o1q0q5w22: 24
- yanzi542766277: 24
- dujijiyiqxx: 17
- wxid_jcnxegjccqi441: 11
- wangchao953541: 9
- ibabaimama: 5
- zhangxiaoyan0422: 5
- snipersk: 5
- wxid_q7vkst94g5u011: 5


# *RegEx for emails and IP addresses / Terms for finding the leaders*

In [14]:
import pandas as pd
import os
import re

# Load chat CSV paths
df_csvs = pd.read_csv("updated_with_csv_paths.csv")

# Patterns
email_pattern = re.compile(r'\S+@\S+\.\S+')
ip_pattern = re.compile(r'\b(?:\d{1,3}\.){3}\d{1,3}(?::\d{1,5})?\b')
role_keywords = [
    "boss"
]
role_pattern = re.compile(r'\b(?:' + '|'.join([re.escape(k) for k in role_keywords]) + r')\b', re.IGNORECASE)

# Store unique matches
seen = set()

# Loop through all files
for csv_path in df_csvs['chats'].dropna().unique():
    if isinstance(csv_path, str) and csv_path.endswith('.csv') and os.path.exists(csv_path):
        try:
            df = pd.read_csv(csv_path)
        except Exception as e:
            print(f"Error reading {csv_path}: {e}")
            continue

        df.columns = [col.lower() for col in df.columns]
        text_cols = [col for col in df.columns if 'message' in col]

        for idx, row in df.iterrows():
            for col in text_cols:
                if pd.notna(row.get(col)):
                    text = str(row[col])

                    for email in email_pattern.findall(text):
                        key = ("email", email, csv_path, idx)
                        if key not in seen:
                            seen.add(key)
                            print(f"[EMAIL] {email} | File: {os.path.basename(csv_path)} | Line: {idx}")

                    for ip in ip_pattern.findall(text):
                        key = ("ip", ip, csv_path, idx)
                        if key not in seen:
                            seen.add(key)
                            print(f"[IP] {ip} | File: {os.path.basename(csv_path)} | Line: {idx}")

                    if role_pattern.search(text):
                        key = ("role", text, csv_path, idx)
                        if key not in seen:
                            seen.add(key)
                            print(f"[ROLE] {text} | File: {os.path.basename(csv_path)} | Line: {idx}")

[ROLE] Boss, let's first look at the one on DingTalk, it's also convenient for matching and corresponding with the departments. | File: 38.csv | Line: 19
[ROLE] It shows that Boss Tong is really awesome. | File: 1.csv | Line: 1862
[ROLE] Oh, acting like a boss now. | File: 1.csv | Line: 2601
[ROLE] Junli's boss has been worn out by accompanying them. | File: 1.csv | Line: 4099
[ROLE] You, as the boss, don't worry about anything else, just focus on performance and revenue. | File: 1.csv | Line: 4285
[ROLE] Do you think they would understand that? They'd probably just think it's a matter of the boss giving a word, so they came directly to us. | File: 1.csv | Line: 4505
[ROLE] The boss can't undermine the financial system. | File: 1.csv | Line: 4508
[ROLE] Their boss won't be back until the end of the month, only the property manager is here, and what he says doesn't count. | File: 1.csv | Line: 6066
[ROLE] You should meet with their boss and communicate, the rent won't be defaulted on, b

In [24]:
def extract_unique_users_from_csv(csv_path):
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f"Error reading {csv_path}: {e}")
        return set()

    # Try to find user-related columns
    user_columns = [col for col in df.columns if col.lower() in ['from', 'to', 'sender', 'receiver', 'user']]
    
    if not user_columns:
        user_columns = df.select_dtypes(include=['object']).columns.tolist()

    # Flatten and clean user list
    users = pd.unique(df[user_columns].values.ravel())
    return {str(u).strip() for u in users if pd.notna(u) and str(u).strip() != ''}

In [25]:
all_users = set()

# Loop through each file path in 'chats' column
for csv_path in df_csvs_llm['chats'].dropna().unique():
    if isinstance(csv_path, str) and csv_path.endswith('.csv') and os.path.exists(csv_path):
        users = extract_unique_users_from_csv(csv_path)
        all_users.update(users)

unique_users_from_chats = sorted(all_users)
print("Unique users found in chat logs:")
for user in unique_users_from_chats:
    print("-", user)

Unique users found in chat logs:
- 25713010771@chatroom
- SWEET5683yao
- Shutd0wn
- adpw90
- dujijiyiqxx
- gzp1991101
- hack05112
- ibabaimama
- just910420
- ken73224
- lengmo
- nullroot
- qq78263462
- snipersk
- tianyi-0608
- wangchao953541
- wei592628
- wxid_12n748um1thl21
- wxid_5390224027312
- wxid_70w3p1jin84k22
- wxid_7p054rmzkhqf21
- wxid_blw54o1q0q5w22
- wxid_c9yv0nsla3yn22
- wxid_hlmnhsq64tt722
- wxid_icges6alg8cl21
- wxid_jcnxegjccqi441
- wxid_kbys0kvzj4ta12
- wxid_mgh25nentc4u22
- wxid_nv9bv435fz3722
- wxid_q7vkst94g5u011
- wxid_soekgggwnfgm21
- wxid_wh6x59w70y3r22
- wxid_xusilpfkh31g21
- wxid_zb45i0rc71yk21
- wxid_zbytkn4qjl3r22
- yanzi542766277
- zhangxiaoyan0422


In [3]:
import os

def count_files_with_os(path):
    return len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])

# Example usage
folder_path = "ocr_to_csv"
file_count = count_files_with_os(folder_path)
print(f"Number of files: {file_count}")

Number of files: 490


# **Financial Infrastructures Analysis**

In [2]:
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
import pandas as pd
import os

In [3]:
import os
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.schema import Document

# Load a single CSV file
csv_file = "csvs/10.csv"
df = pd.read_csv(csv_file)

# Initialize the embedding model
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

# Setup vector DB path
db_location = "./chroma_financial_chat_db"
add_documents = not os.path.exists(db_location)

# Only add documents if the database doesn't already exist
if add_documents:
    documents = []
    ids = []
    for i, row in df.iterrows():
        document = Document(
            page_content=row["message_translation"],
            metadata={
                "sender": row["From"],
                "receiver": row["To"],
                "timestamp": row["Time"],
                "source_file": os.path.basename(csv_file),   # Track source file
                "line_number": i + 1                         # Track line number
            },
            id=f"{os.path.basename(csv_file)}_{i}"            # Ensure unique ID
        )
        documents.append(document)
        ids.append(f"{os.path.basename(csv_file)}_{i}")

# Create or load the Chroma vector store
vector_store = Chroma(
    collection_name="financial_chats",
    persist_directory=db_location,
    embedding_function=embeddings
)

if add_documents:
    vector_store.add_documents(documents=documents, ids=ids)

# Setup retriever for use in RAG
retriever = vector_store.as_retriever(search_kwargs={"k": 15})

/tmp/ipykernel_20294/3807150908.py:38: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vector_store = Chroma(


In [4]:
model = OllamaLLM(model="gemma3:27b") 

template = """
You are a cybersecurity analyst reviewing translated chat messages for financial intelligence.

Your task is to extract ONLY the messages that contain **financially relevant information**, such as:

- Payments or money transfers
- Amounts, currencies, or sums of money
- Payment methods (cash, crypto, bank, wallets)
- Account numbers, codes, or financial identifiers
- Mentions of financial roles (payer, payee, client, broker)
- References to suspicious or recurring financial activity

From the messages below:
{chats}

Return ONLY the messages that meet any of the criteria above.

For each relevant message, output:
- The **timestamp** (if provided in the metadata or inferred)
- The **translated content** of the message

Do not summarize. Do not explain. Do not include non-financial content.  
**DO NOT create or infer messages. Use only the exact text from the messages provided. If no messages match, return nothing.**

Output format:
[Time] Message
"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

In [5]:
question = "Extract only the chat messages that refer to money, payments, currencies, accounts, financial identifiers, or financial discussions"

# Retrieve relevant chats
retrieved_docs = retriever.invoke(question)

# Combine messages with metadata for context
chat_texts = "\n\n".join([
    f"[{doc.metadata['source_file']} - line {doc.metadata['line_number']}] {doc.page_content}"
    for doc in retrieved_docs
])

# Run through the LLM
response = chain.invoke({"chats": chat_texts})

# Output the result
print(response)

[10.csv - line 21] Account TGtadie, phone: 18510867099, name: Wang Ning.






In [6]:
import pandas as pd
import os

In [13]:
import pandas as pd
import os
from langchain.llms import Ollama as OllamaLLM
from langchain.prompts import ChatPromptTemplate

# === Step 1: Define the LLM and prompt chain ===
model = OllamaLLM(model="gemma3:27b")
prompt = ChatPromptTemplate.from_template("{chats}")
chain = prompt | model

# === Step 2: Define function that takes any retrieved doc ===
def investigate_target_message(doc, chain, context_window=15, base_folder="csvs"):
    # Get metadata
    file = doc.metadata["source_file"]
    line = doc.metadata["line_number"]
    filepath = os.path.join(base_folder, file)

    # Load the CSV
    if not os.path.exists(filepath):
        print(f"File not found: {filepath}")
        return None

    df = pd.read_csv(filepath)
    start = max(1, line - context_window)
    end = min(len(df), line + context_window)

    # Build context window
    context_lines = []
    for i in range(start, end + 1):
        row_text = str(df.iloc[i - 1]["message_translation"])
        tag = "**[TARGET]**" if i == line else ""
        context_lines.append(f"[{file} - line {i}]{tag} {row_text}")

    full_context = "\n".join(context_lines)

    # Construct prompt
    prompt_text = f"""
You are a financial crime investigator analyzing financial communications.

The following messages include a suspicious message marked **[TARGET]** and 30 nearby messages for context. Based on this, determine:

- The financial structure implied (e.g., payer, broker, client, intermediary)
- Entities involved (names, accounts, phones)
- Any suspicious behavior or coordinated activity
- Recommended follow-up steps

Messages:
{full_context}

Generate a structured intelligence report based on the **[TARGET]** message.
"""

    # Run LLM
    return chain.invoke({"chats": prompt_text})

# === Step 3: Example usage ===
# Run on any retrieved document
doc = retrieved_docs[0]  # or loop over retrieved_docs
response = investigate_target_message(doc, chain)
print(response)

## Intelligence Report - Potential Illicit Software/Service Procurement

**Date:** October 26, 2023
**Subject:** Potential Procurement of WhatsApp Exploitation/Access Services
**Source:** Financial Communication Analysis - 10.csv (Lines 107-137)
**Analyst:** Financial Crime Investigator

**1. Executive Summary:**

This report details a concerning exchange indicating potential interest in acquiring WhatsApp access or exploitation capabilities. The conversation centers around inquiries about "WhatsApp products" and suggests a willingness to pay for such services, potentially for illicit purposes. The emphasis on meeting in person, coupled with references to a "leader" and securing a "project," suggests a coordinated effort and potentially larger scale operation.

**2. Financial Structure & Entities Involved:**

* **Mr. Zhou:**  Represents a company that *claims* to not offer WhatsApp services, but is being actively probed about them. Position unclear, but appears to be a potential interm